In [3]:
!pip install scipy

     --------------------------------------- 42.5/42.5 MB 22.6 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import bernoulli, binom
import math


In [10]:
mu_a = 0.5
X_a = bernoulli(mu_a)
#mu_as
X_as = bernoulli(0.5)
X_a.rvs(10)

array([1, 1, 1, 0, 0, 0, 1, 0, 1, 1], dtype=int64)

In [106]:
T = 2000
mu_1 = 0.5 - 1/4
mu_2 = 0.5 - 1/8
mu_3 = 0.5 - 1/16


X = bernoulli(mu_1)
sample1 = X.rvs(2000)
X = bernoulli(mu_2)
sample2 = X.rvs(2000)
X = bernoulli(mu_3)
sample3 = X.rvs(2000)
arms = np.array([sample1, sample2, sample3])

K = len(arms)


In [115]:
def hedge_alg(eta0, arms, reparametrized = False, anytime = False):
    K = len(arms)
    L_prev = [0 for arm in arms ]
    played_arm = np.array([])

    for t in range(T):
        if anytime: eta = eta0[t] 
        else: eta = eta0

        #p_t = float x 3
        p_t = [(math.exp(-eta * L_prev[arm]) / sum([math.exp(-eta * L_tminus1) for L_tminus1 in L_prev])) for arm in range(len(arms))]
        
        #Sample A_t according to p_t and play it
        A_t = p_t.index(max(p_t))
        played_arm = np.append(played_arm, A_t)

        #update L_t
        L_prev = [L_prev[a] + arms[a][t].tolist() for a in range(len(L_prev))]

    return played_arm


In [130]:
eta_h = math.sqrt((2 * math.log(K))/T)
eta_rp = math.sqrt((8 * math.log(K))/T)
eta_anytime_h = [math.sqrt((math.log(K))/t) for t in range(1, T+1)]
eta_anytime_ep = [2 * math.sqrt((math.log(K))/t) for t in range(1, T+1)]

out_h = hedge_alg(eta_h, arms)
out_rp = hedge_alg(eta_rp, arms, reparametrized = True)
out_anytime_h = hedge_alg(eta_anytime_h, arms, anytime = True)


print(out_h[0:20])
print(out_anytime_h[0:20])

[0. 0. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [135]:

print(np.count_nonzero(out_anytime_h == 1))
print(np.count_nonzero(out_h == 1))

4
4
